# 

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [1]:
import keras as K

Using TensorFlow backend.
F:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1282061847465075206
]


In [11]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range
import math
def euDis(c1,c2):
    return math.sqrt((c1[0]-c2[0])**2+(c1[1]-c2[1])**2)

In [12]:
HER_GRID = 50
VER_GRID = 50
HER_LEN = 1050
VER_LEN = 750
a_col = 6
a_row = 4
hopDis = 2
COL = HER_LEN//HER_GRID
ROW = VER_LEN//VER_GRID
def grid_anchors(rowCells,colCells,row=4,col=6):
    st_w=colCells//(col+1)
    st_h=rowCells//(row+1)
    #print(st_w,st_h)
    ini_w=0
    ini_h=0
    anchors = []
    for i in range(row):
        ini_h+=st_h
        ini_w=0
        for j in range(col):
            ini_w+=st_w
            anchors.append([ini_w-1,ini_h-1])
    return anchors

anchors = grid_anchors(ROW,COL)
PRElOCATION=10
hoplimits = 10

In [13]:
class GridPoint():
    def __init__(self, pStr):
        pStrList = pStr.split(" ")
        self.point = [int(x) for x in pStrList]
        self.x = self.point[0]
        self.y = self.point[1]
        
def encodeGPList(gpList, nRows):
    x = np.zeros((nRows, COL+ROW))
    #print(len(gpList))
    for i, p in enumerate(gpList):
        x[i, p.x] = 1
        yy = COL+p.y
        if(yy>=COL+ROW):
            yy=COL+ROW-1
        x[i, yy] = 1
    return x

def decodeToGP(oneHot):
    gpList = []
    for row in oneHot[0]:
        count = 0
        cur = []
        first = True
        #print(row)
        for cell in row:
            
            
            if cell ==1:
                if not first:
                    count-=COL
                first = False
                cur.append(count)
            count+=1
        gpList.append(cur)
    return gpList


class constraint():
    def __init__(self,consStr,gpList):
        self.gpList = gpList
        self.hopInfo=[]
        if consStr == "#": 
            self.hopInfo = [1,0,0,0,0,0,0]
        elif consStr == "*": 
            self.hopInfo = [0,1,0,0,0,0,0]
        else:
            cStrList = consStr.replace("(","").replace(")","").split(";")[:5]
            self.hopInfo = [0,0]+[int(float(x)) for x in cStrList]

    def encodeConstraints(self,anchors,hopLimits):
        col = max(len(anchors),hopLimits)
        x = np.zeros((5, col))
        for i, p in enumerate(self.hopInfo[2:]):
            x[i, p] = 1
        
        return x
        
    def findFirstLastAppear(self):
        leftAnchor = anchors[self.hopInfo[2]]
        rightAnchor = anchors[self.hopInfo[3]]
        circleCenter = anchors[self.hopInfo[4]]
        x=circleCenter[0]
        y=circleCenter[1]
        r=self.hopInfo[6]*hopDis
        a=self.hopInfo[5]*hopDis
        a2=a-hopDis
        
        h1x=leftAnchor[0]
        h1y=leftAnchor[1]
        h2x=rightAnchor[0]
        h2y=rightAnchor[1]

        rr=r*r 
        res = [-1,-1]
        for k,tup in enumerate(self.gpList):
            i=tup.x
            j=tup.y
            di=x-i
            dj=y-j
            if  (di*di+dj*dj<=rr and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))<=2*a and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))>=2*a2):
                res[0] = k
                break
        else: res[0] = k
        for k,tup in enumerate(self.gpList[::-1]):
            i=tup.x
            j=tup.y
            di=x-i
            dj=y-j
            if  (di*di+dj*dj<=rr and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))<=2*a and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))>=2*a2):
                res[1] = len(self.gpList)  - k 
                break
        else: res[1] = k 
        #print(res)
        return res
    
    def myEncoderInput(self,nRows):
        first,last = self.findFirstLastAppear()
        gpList = []
        
        if last-first>=nRows:
            gpList=self.gpList[first:first+nRows]
        else:
            if last-nRows>=0:
                gpList=self.gpList[last-nRows:last]
            else:
                gpList = self.gpList[:min(len(self.gpList),nRows)]
                
        self.encoderInput = encodeGPList(gpList,nRows)
        return self.encoderInput
        
    def decodeConstraint (self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
            
        return [x for x in x]

In [14]:
f = open("./pacmanTrain.txt", "r")

stops = f.readline()
for i in range(int(stops)):
    print(f.readline())
print(f.readline())

12 1

11 1

10 1

9 1

8 1

8 2

8 3

8 4

8 5

7 5

8 5

9 5

9 4

9 3

8 3

8 2

8 1

(23; 1; 2; 3.000000; 1.000000; 2.000000; 2.000000)



In [15]:
f = open("./pacmanTrain.txt", "r")
TAS_SIZE = int(f.readline())
gpList = []
for i in range(TAS_SIZE):
    gpList.append(GridPoint(f.readline()))
print([p.point for p in gpList])
firstConstraint = constraint(f.readline(),gpList)
f.close()

print(firstConstraint.hopInfo)
decoderInput = firstConstraint.encodeConstraints(anchors,hoplimits)
print(decoderInput)





[[12, 1], [11, 1], [10, 1], [9, 1], [8, 1], [8, 2], [8, 3], [8, 4], [8, 5], [7, 5], [8, 5], [9, 5], [9, 4], [9, 3], [8, 3], [8, 2], [8, 1]]
[0, 0, 23, 1, 2, 3, 1]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [9]:

encoderInput = firstConstraint.myEncoderInput(PRElOCATION)
print(encoderInput)
print(encoderInput[0])







[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [17]:
import os.path
from os import path

strokeRange = [20,194]
totalNumOfTestCases = 0
f = open("./pacmanTrain.txt", "r")

while(True):
    TAS_SIZE = f.readline()
    
    if not TAS_SIZE: break
    TAS_SIZE = int(TAS_SIZE)    
    totalNumOfTestCases+=1    
    for i in range(TAS_SIZE):
        f.readline()
    f.readline()

f.close()



print (totalNumOfTestCases)

21180


In [18]:
import os.path
from os import path


x = np.zeros((totalNumOfTestCases, PRElOCATION, ROW+COL), dtype=np.bool)
y = np.zeros((totalNumOfTestCases, 5, max(a_col*a_row,hoplimits)), dtype=np.bool)

ttc=0

f = open("./pacmanTrain.txt", "r")

while(True):
    TAS_SIZE = f.readline()
    
    if not TAS_SIZE: break
    TAS_SIZE = int(TAS_SIZE)    
    totalNumOfTestCases+=1 
    gpList = []
    for i in range(TAS_SIZE):
        gpList.append(GridPoint(f.readline()))
    firstConstraint = constraint(f.readline(),gpList)
    y[ttc] = firstConstraint.encodeConstraints(anchors,hoplimits)
    x[ttc] = firstConstraint.myEncoderInput(PRElOCATION)
    ttc+=1
f.close()

# for sr in range(strokeRange[0],strokeRange[1]):
#     for item in range(1,100000):
#         fileName = "./dataExpect/"+str(sr)+"/preTrain_"+str(item)
#         if path.exists(fileName):
#             f = open(fileName, "r")
#             TAS_SIZE = int(f.readline())
#             gpList = []
#             for i in range(TAS_SIZE):
#                 gpList.append(GridPoint(f.readline()))
#             firstConstraint = constraint(f.readline(),gpList)
#             f.close()
#             y[ttc] = firstConstraint.encodeConstraints(anchors,hoplimits)
#             x[ttc] = firstConstraint.myEncoderInput(PRElOCATION)
#             ttc+=1

In [19]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [20]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [21]:
print('Training Data:')
print(x_train.shape)
print(y_train.shape)


Training Data:
(19062, 10, 36)
(19062, 5, 24)


In [22]:
print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Validation Data:
(2118, 10, 36)
(2118, 5, 24)


In [23]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 256
BATCH_SIZE = 128
LAYERS = 1

In [27]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(PRElOCATION, ROW+COL)))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.

model.add(layers.RepeatVector(5))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(24, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

'''

encoder_inputs = Input(shape=(None, 50))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 5))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(5, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


'''




Build model...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               300032    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 24)             6168      
Total params: 831,512
Trainable params: 831,512
Non-trainable params: 0
_________________________________________________________________


"\n\nencoder_inputs = Input(shape=(None, 50))\nencoder = LSTM(256, return_state=True)\nencoder_outputs, state_h, state_c = encoder(encoder_inputs)\n# We discard `encoder_outputs` and only keep the states.\nencoder_states = [state_h, state_c]\n\n# Set up the decoder, using `encoder_states` as initial state.\ndecoder_inputs = Input(shape=(None, 5))\n# We set up our decoder to return full output sequences,\n# and to return internal states as well. We don't use the \n# return states in the training model, but we will use them in inference.\ndecoder_lstm = LSTM(5, return_sequences=True, return_state=True)\ndecoder_outputs, _, _ = decoder_lstm(decoder_inputs,\n                                     initial_state=encoder_states)\ndecoder_dense = Dense(num_decoder_tokens, activation='softmax')\ndecoder_outputs = decoder_dense(decoder_outputs)\n\n# Define the model that will turn\n# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`\nmodel = Model([encoder_inputs, decoder_inpu

In [28]:

# def ga(rowCells,colCells,row=5,col=8):
#     st_w=colCells//(col+1)
#     st_h=rowCells//(row+1)
#     #print(st_w,st_h)
#     ini_w=0
#     ini_h=0
#     anchors = {}
#     count = 0
#     for i in range(row):
#         ini_h+=st_h
#         ini_w=0
#         for j in range(col):
#             ini_w+=st_w
#             anchors[count] = (ini_w,ini_h)
#             count+=1
#     return anchors

# aac = ga(60,90)
# aac

In [29]:
# Train the model each generation and show predictions against the validation
# dataset.

class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'
    
    
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        correct = firstConstraint.decodeConstraint(rowy[0])
        guess = firstConstraint.decodeConstraint(preds[0], calc_argmax=False)
        
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 17s 870us/step - loss: 1.1961 - accuracy: 0.6439 - val_loss: 0.4116 - val_accuracy: 0.8672
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [0, 16, 21, 3, 1] ☒ [5, 13, 20, 3, 1]
T [4, 6, 12, 3, 1] ☑ [4, 6, 12, 3, 1]
T [23, 6, 7, 3, 1] ☑ [23, 6, 7, 3, 1]
T [18, 10, 9, 3, 1] ☑ [18, 10, 9, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [18, 10, 3, 3, 1] ☑ [18, 10, 3, 3, 1]
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]

--------------------------------------------------
Iteration 2
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 825us/step - loss: 0.2874 - accuracy: 0.9149 - val_loss: 0.2053 - val_accuracy

19062/19062 [==============================] - 15s 812us/step - loss: 0.0906 - accuracy: 0.9696 - val_loss: 0.0875 - val_accuracy: 0.9722
T [23, 2, 8, 3, 1] ☑ [23, 2, 8, 3, 1]
T [5, 12, 8, 4, 4] ☑ [5, 12, 8, 4, 4]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [5, 13, 19, 3, 1] ☑ [5, 13, 19, 3, 1]
T [23, 2, 8, 3, 1] ☑ [23, 2, 8, 3, 1]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [4, 6, 0, 3, 1] ☑ [4, 6, 0, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [4, 6, 12, 3, 1] ☑ [4, 6, 12, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]

--------------------------------------------------
Iteration 13
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 15s 807us/step - loss: 0.0865 - accuracy: 0.9708 - val_loss: 0.0899 - val_accuracy: 0.9705
T [18, 10, 9, 3, 1] ☑ [18, 10, 9, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [0, 16, 10, 3, 1] ☑ [0, 16, 10, 3, 1]
T [0, 10, 22, 3, 1] ☑ [0, 10, 22, 3, 1]
T [0, 10, 22, 3, 1

19062/19062 [==============================] - 15s 805us/step - loss: 0.0534 - accuracy: 0.9795 - val_loss: 0.0822 - val_accuracy: 0.9727
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [17, 0, 19, 4, 4] ☑ [17, 0, 19, 4, 4]
T [23, 2, 8, 3, 1] ☒ [23, 1, 2, 3, 1]
T [0, 22, 21, 3, 1] ☑ [0, 22, 21, 3, 1]
T [5, 7, 13, 3, 1] ☒ [5, 7, 21, 3, 1]
T [18, 10, 9, 3, 1] ☑ [18, 10, 9, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]

--------------------------------------------------
Iteration 35
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 817us/step - loss: 0.0515 - accuracy: 0.9801 - val_loss: 0.0690 - val_accuracy: 0.9747
T [4, 6, 12, 3, 1] ☑ [4, 6, 12, 3, 1]
T [23, 6, 7, 3, 1] ☑ [23, 6, 7, 3, 1]
T [11, 13, 19, 3, 1] ☒ [11, 13, 19, 4, 4]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [18, 3, 9, 3, 1] ☑ [18, 3, 9, 3, 1]
T [18, 10, 3, 3, 1] 

19062/19062 [==============================] - 16s 854us/step - loss: 0.0430 - accuracy: 0.9826 - val_loss: 0.0786 - val_accuracy: 0.9753
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [23, 13, 7, 3, 1] ☑ [23, 13, 7, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [23, 2, 8, 3, 1] ☑ [23, 2, 8, 3, 1]

--------------------------------------------------
Iteration 57
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 846us/step - loss: 0.0437 - accuracy: 0.9823 - val_loss: 0.0742 - val_accuracy: 0.9757
T [0, 10, 16, 3, 1] ☑ [0, 10, 16, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [23, 1, 2, 3, 1] ☑ [23, 1, 2, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [0, 10, 22, 3, 1] ☑ [0, 10, 22, 3, 1]
T [18, 11, 2, 4, 4] ☑ 

19062/19062 [==============================] - 16s 844us/step - loss: 0.0392 - accuracy: 0.9832 - val_loss: 0.0790 - val_accuracy: 0.9752
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [17, 0, 19, 4, 4] ☑ [17, 0, 19, 4, 4]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [18, 10, 3, 3, 1] ☑ [18, 10, 3, 3, 1]
T [5, 20, 14, 3, 1] ☑ [5, 20, 14, 3, 1]
T [18, 10, 9, 3, 1] ☑ [18, 10, 9, 3, 1]
T [5, 7, 13, 3, 1] ☒ [0, 7, 13, 3, 1]
T [5, 7, 13, 3, 1] ☑ [5, 7, 13, 3, 1]

--------------------------------------------------
Iteration 79
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 841us/step - loss: 0.0390 - accuracy: 0.9837 - val_loss: 0.0785 - val_accuracy: 0.9746
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [18, 10, 9, 3, 1] ☑ [18, 10, 9, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [0, 10, 22, 3, 1] ☑ [0, 10, 22, 3, 1]
T [5, 20, 14, 3, 1] ☑ [5, 20, 14, 3, 1]
T [18, 10,

19062/19062 [==============================] - 16s 842us/step - loss: 0.0376 - accuracy: 0.9842 - val_loss: 0.0813 - val_accuracy: 0.9758
T [23, 6, 7, 3, 1] ☑ [23, 6, 7, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [18, 10, 3, 3, 1] ☑ [18, 10, 3, 3, 1]
T [12, 4, 3, 3, 1] ☑ [12, 4, 3, 3, 1]
T [0, 16, 21, 3, 1] ☑ [0, 16, 21, 3, 1]
T [23, 1, 2, 3, 1] ☑ [23, 1, 2, 3, 1]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]

--------------------------------------------------
Iteration 101
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 842us/step - loss: 0.0379 - accuracy: 0.9835 - val_loss: 0.0808 - val_accuracy: 0.9764
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [4, 6, 12, 3, 1] ☑ [4, 6, 12, 3, 1]
T [23, 1, 2, 3, 1] ☑ [23, 1, 2, 3, 1]
T [0, 16, 15, 3, 1] ☑ [0, 16, 15, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [23, 6, 3, 4, 4] ☑ 

19062/19062 [==============================] - 16s 838us/step - loss: 0.0381 - accuracy: 0.9839 - val_loss: 0.0861 - val_accuracy: 0.9749
T [17, 0, 19, 4, 4] ☑ [17, 0, 19, 4, 4]
T [18, 10, 9, 3, 1] ☑ [18, 10, 9, 3, 1]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [23, 1, 2, 3, 1] ☑ [23, 1, 2, 3, 1]
T [0, 10, 22, 3, 1] ☒ [0, 17, 20, 4, 4]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [23, 0, 3, 4, 4] ☑ [23, 0, 3, 4, 4]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [0, 16, 21, 3, 1] ☑ [0, 16, 21, 3, 1]

--------------------------------------------------
Iteration 123
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 841us/step - loss: 0.0384 - accuracy: 0.9840 - val_loss: 0.0876 - val_accuracy: 0.9747
T [23, 6, 7, 3, 1] ☑ [23, 6, 7, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [0, 16, 21, 3, 1] ☒ [5, 13, 20, 3, 1]
T [17, 0, 19, 4, 4] ☑ [17, 0, 19, 4, 4]
T [5, 13, 2

19062/19062 [==============================] - 16s 841us/step - loss: 0.0366 - accuracy: 0.9838 - val_loss: 0.0845 - val_accuracy: 0.9752
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]
T [12, 10, 4, 3, 1] ☑ [12, 10, 4, 3, 1]
T [18, 10, 3, 3, 1] ☑ [18, 10, 3, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [18, 10, 9, 3, 1] ☑ [18, 10, 9, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]

--------------------------------------------------
Iteration 145
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 838us/step - loss: 0.0360 - accuracy: 0.9843 - val_loss: 0.0818 - val_accuracy: 0.9770
T [23, 2, 8, 3, 1] ☑ [23, 2, 8, 3, 1]
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]
T [11, 13, 19, 3, 1] ☑ [11, 13, 19, 3, 1]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [23, 6, 7, 3, 1] ☑ [23, 6, 7, 3, 1]
T [23, 6,

19062/19062 [==============================] - 16s 834us/step - loss: 0.0359 - accuracy: 0.9844 - val_loss: 0.0884 - val_accuracy: 0.9764
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]
T [0, 17, 20, 4, 4] ☑ [0, 17, 20, 4, 4]
T [4, 12, 6, 3, 1] ☑ [4, 12, 6, 3, 1]
T [11, 1, 7, 3, 1] ☑ [11, 1, 7, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]

--------------------------------------------------
Iteration 167
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 835us/step - loss: 0.0358 - accuracy: 0.9844 - val_loss: 0.0881 - val_accuracy: 0.9763
T [4, 6, 12, 3, 1] ☑ [4, 6, 12, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [23, 6, 7, 3, 1] ☑ [23, 6, 7, 3, 1]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [11, 13, 19, 3, 1] ☑ [11, 13, 19, 3, 1]
T [4, 6, 18, 3, 1] ☑ [4

19062/19062 [==============================] - 16s 844us/step - loss: 0.0354 - accuracy: 0.9845 - val_loss: 0.0908 - val_accuracy: 0.9742
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [0, 10, 22, 3, 1] ☑ [0, 10, 22, 3, 1]
T [11, 13, 19, 3, 1] ☒ [11, 12, 19, 3, 4]
T [5, 13, 20, 3, 1] ☑ [5, 13, 20, 3, 1]
T [23, 6, 3, 4, 4] ☑ [23, 6, 3, 4, 4]
T [5, 7, 0, 3, 1] ☑ [5, 7, 0, 3, 1]
T [5, 12, 8, 4, 4] ☑ [5, 12, 8, 4, 4]
T [0, 10, 22, 3, 1] ☑ [0, 10, 22, 3, 1]
T [23, 2, 8, 3, 1] ☑ [23, 2, 8, 3, 1]

--------------------------------------------------
Iteration 189
Train on 19062 samples, validate on 2118 samples
Epoch 1/1
19062/19062 [==============================] - 16s 836us/step - loss: 0.0351 - accuracy: 0.9841 - val_loss: 0.0913 - val_accuracy: 0.9751
T [4, 12, 6, 3, 1] ☑ [4, 12, 6, 3, 1]
T [11, 1, 7, 3, 1] ☑ [11, 1, 7, 3, 1]
T [5, 12, 8, 4, 4] ☑ [5, 12, 8, 4, 4]
T [4, 6, 18, 3, 1] ☑ [4, 6, 18, 3, 1]
T [18, 10, 3, 3, 1] ☑ [18, 10, 3, 3, 1]
T [4, 6, 18, 3, 1] 

In [31]:
import pickle
filename='model2.pkl'
pickle.dump(model, open(filename, 'wb'))

In [31]:
import time
ind = np.random.randint(0, len(x_val))
rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
start_time = time.time()
preds = model.predict_classes(rowx, verbose=0)


correct = firstConstraint.decodeConstraint(rowy[0])
guess = firstConstraint.decodeConstraint(preds[0], calc_argmax=False)
print(rowx.shape)
print(decodeToGP(rowx))
print('T', correct, end=' ')
if correct == guess:
    print(colors.ok + '☑' + colors.close, end=' ')
else:
    print(colors.fail + '☒' + colors.close, end=' ')
print(guess)
print("--- %s seconds ---" % (time.time() - start_time))


(1, 50, 150)
[[49, 28], [48, 28], [49, 29], [48, 29], [48, 30], [47, 30], [47, 31], [46, 31], [47, 32], [45, 31], [46, 32], [45, 32], [44, 32], [45, 33], [44, 33], [43, 33], [42, 33], [41, 33], [40, 33], [40, 32], [40, 31], [40, 30], [40, 29], [40, 28], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
T [38, 35, 3, 3, 8] ☑ [38, 35, 3, 3, 8]
--- 0.07047009468078613 seconds ---


In [52]:
%gui tk
import tkinter as tk
import time

ANCHORS={}
HER_GRID = 10
VER_GRID = 10
HER_LEN = 900
VER_LEN = 600

class myTk():
    def __init__(self,root,gridTrajectory,correct,guess):
        self.root = root
        self.canvasSize = [HER_LEN,VER_LEN]
        #self.gridStep=20
        self.c = tk.Canvas(root, height=self.canvasSize[1], width=self.canvasSize[0], bg='white')
        self.c.pack()
        self.button = tk.Button(master=root, text="Quit", command=self._quit)
        self.button.pack(side=tk.BOTTOM)
        self.row = 60
        self.col = 90
        self.create_grid()
        self.anchors = self.create_grid_anchors()
        #self.plot_trajectory(gridTrajectory)
        
    def allButtons(self,root):
        self.allButtonsFrame = tk.Frame(root)
        
    
    
    def create_grid(self,event=None):
        w = self.canvasSize[0] # Get current width of canvas
        h = self.canvasSize[1] # Get current height of canvas
        self.c.delete('grid_line') # Will only remove the grid_line

        # Creates all vertical lines at intevals of 100
        for i in range(0, w, HER_GRID):
            self.c.create_line([(i, 0), (i, h)], tag='grid_line')

        # Creates all horizontal lines at intevals of 100
        for i in range(0, h, VER_GRID):
            self.c.create_line([(0, i), (w, i)], tag='grid_line')

    def create_grid_anchors(self, row=5,col=8):
        w = self.canvasSize[0] # Get current width of canvas
        h = self.canvasSize[1] # Get current height of canvas
        st_w=w//(col+1)
        st_h=h//(row+1)
        print(st_w,st_h)
        ini_w=0
        ini_h=0
        anchors = {}
        count=0
        for i in range(row):
            ini_h+=st_h
            ini_w=0
            for j in range(col):
                ini_w+=st_w
                #self.c.create_rectangle(ini_w, ini_h, ini_w+HER_GRID, ini_h+VER_GRID,fill="red",tag='anchors')
                anchors[count]=(ini_w//st_w,ini_h//st_h)
                count+=1
        return anchors
    
    def plot_correct(self,correct):
        h1 = self.anchors[correct[0]]
        h2 = self.anchors[correct[1]]
        c = self.anchors[correct[2]]
        a = correct[3]
        r = correct[4]
        for i in range(self.row):
            for j in range(self.col):
                
                
    def plot_trajectory(self, gridTrajectory):
        for grid in gridTrajectory:
            time.sleep(0.1)
            self.root.update()
            ini_w = grid[0]*HER_GRID
            ini_h = grid[1]*VER_GRID
            print(ini_w,ini_h)
            self.c.create_rectangle(ini_w, ini_h, ini_w+HER_GRID, ini_h+VER_GRID,fill="green",tag='trajectory')

    def _quit(self):
        root.quit()     # stops mainloop
        root.destroy()  # this is necessary on Windows to prevent
                        # Fatal Python Error: PyEval_RestoreThread: NULL tstate


root = tk.Tk()
my_gui = myTk(root,decodeToGP(rowx),correct,guess)
root.mainloop()

100 100
320 320
310 310
320 310
330 310
320 300
340 310
330 300
340 300
350 300
360 300
370 300
360 290
380 300
370 290
380 290
390 290
400 290
410 290
420 290
430 290
420 280
430 280
440 290
450 290
440 300
450 300
460 300
450 310
470 300
460 310
470 310
480 310
480 320
490 320
480 330
490 330
500 330
500 320
510 320
500 310
510 310
520 310
510 300
530 310
520 300
530 300
540 310
530 320
540 320
550 320
